In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip


In [16]:
file = open('amazon-meta.txt', encoding ='latin-1') 
lines = file.read().split('\n\n')

In [18]:
# to see the data
lines[2].split('\n')

['Id:   1',
 'ASIN: 0827229534',
 '  title: Patterns of Preaching: A Sermon Sampler',
 '  group: Book',
 '  salesrank: 396585',
 '  similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X',
 '  categories: 2',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]',
 '  reviews: total: 2  downloaded: 2  avg rating: 5',
 '    2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9',
 '    2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5']

In [7]:
#converting it into a dictionary
d = {'Id':[], 'ASIN':[], 'title':[], 'group':[], 'salesrank':[], 'similar':[], 'categories':[], 'reviews':[]}

def split_line(line):
    """
    Helper function to split line on colon and return name and value.
    """
    name = line.split(': ')[0].strip()
    
    #Check if name is in the keys of the master dictionary
    if name not in d.keys():
        return 0
    try:
        value = line.split(': ')[1:len(line.split(': '))]      
    #IndexError if splitting on line like "discontinued product"
    except IndexError:
        return 0
    return (name, value)  

def update_master_dict(line_dict):
    """
    Helper function to update master dictionary with a completed row.
    """
    for key in d.keys():
        d[key].append(line_dict[key])
        
status = 0
id_dict = {}

with open('amazon-meta.txt', encoding ='latin-1') as f:
    for line in f.read().splitlines():
        #line = line.decode("latin-1")
        
        #If name, value pair returned and status is not looking for multi-lines
        if split_line(line) != 0 and status == 0:
            (name, value) = split_line(line)
            if name == 'Id':
                #Initialize row dictionary
                id_dict = {name: value[0].strip()} 
            if 'categories' in name:
                #Determine number of following rows to include
                rows = int(value[0])
                if rows > 0:
                    (i, status, compiled_rows) = (0, 1, [])
                else:
                    #If no categories returned, set value to 0
                    id_dict[name] = 0  
            if 'reviews' in name:
                #Determine number of following rows to include
                rows = int(value[2].split()[0])
                if rows > 0:
                    (i, status, compiled_rows) = (0, 1, [])
                else:
                    #If no categories returned, set value to 0
                    id_dict[name] = 0 
            else:
                #If any other column besides reviews, categories or ID, just add value
                id_dict[name] = value[0].strip()
                
        #If collecting multi-lines
        elif status == 1:
            compiled_rows.append(line.strip())
            i += 1
            #If collected all lines to match number of rows, add lines as value and reset status
            if i == rows:
                id_dict[name] = compiled_rows
                status = 0 
        else:
            #If reached end of row, reset status to looking for next ID
            status = 0
            
            #If entire row collected, add to master dictionary
            if len(id_dict) == len(d):
                update_master_dict(id_dict)
            id_dict = {}

#Create dataframe from master dictionary          
df = pd.DataFrame.from_dict(d) 
df.head()

,ASIN,Id,categories,group,reviews,salesrank,similar,title
0,0827229534,1,[|Books[283155]|Subjects[1000]|Religion & Spir...,Book,[2000-7-28 cutomer: A2JW67OY8U6HHK rating: 5...,396585,5 0804215715 156101074X 0687023955 0687074...,Patterns of Preaching
1,0738700797,2,[|Books[283155]|Subjects[1000]|Religion & Spir...,Book,[2001-12-16 cutomer: A11NCO6YTE4BTJ rating: ...,168596,5 0738700827 1567184960 1567182836 0738700...,Candlemas
2,0486287785,3,[|Books[283155]|Subjects[1000]|Home & Garden[4...,Book,[2003-7-10 cutomer: A3IDGASRQAW8B2 rating: 5...,1270652,0,World War II Allied Fighter Planes Trading Cards
3,0842328327,4,[|Books[283155]|Subjects[1000]|Religion & Spir...,Book,[2004-8-19 cutomer: A2591BUPXCS705 rating: 4...,631289,5 0842328130 0830818138 0842330313 0842328...,Life Application Bible Commentary
4,1577943082,5,[|Books[283155]|Subjects[1000]|Religion & Spir...,Book,0,455160,5 157794349X 0892749504 1577941829 0892749...,Prayers That Avail Much for Business


# Algorithm Thinking

For the categorization, one need to build the algorithm from scratch and carefully avoids the overlaping, In case of overlaping
there should not be much redundancy.

# Product Thinking

1) Yes, it is a good idea to show the hierarchy for items, it will help the consumer look for products easily.   
      
2) Yes, it is good to show the simiar items because, consumers will have more options to purchase similar products. It is also 
a good way to do the marketing.

3)Yes, showing the reviews and ratings will give consumers full confidence while making a purchase, if reviews and ratings are 
missing, the consumer might be skeptical about making a purchase. So it is better to provide the ratings and reviews.     
    
4) We can establish all of the above by doing an A/B testing. It will validate the usage.